# **Analísis de sentimiento**

## **Sebastián Carrero Cardona**

# **Objetivo:**

Realizar un análisis de sentimiento en una base de datos que contiene comentarios sobre varios juegos, cada uno de los cuales ya cuenta con una etiqueta de sentimiento predefinida. Luego, construir una tabla comparativa de la precisión obtenida mediante la evaluación de cuatro grupos de comentarios aleatorios en dos etapas. En la primera etapa, aplicaremos el análisis de sentimiento a cada grupo de comentarios sin realizar ningún tipo de preprocesamiento, mientras que en la segunda etapa aplicaremos preprocesamiento antes del análisis.


## **Importación de librerías**

In [1]:
import pandas as pd
import random
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import unicodedata
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

c:\Users\Sebastián Carrero\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Sebastián Carrero\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Sebastián Carrero\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to C:\Users\Sebastián
[nltk_data]     Carrero\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sebastián
[nltk_data]     Carrero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Cargamos dataset con los comentarios**

In [16]:
df = pd.read_csv('../PROCESAMIENTO_LENGUAJE_NATUAL/comentarios.csv')
df

,Registro,Juego,Sentimiento,Comentario
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [17]:
#Converción de todos los comentarios a tipo string
df['Comentario'] = df['Comentario'].astype(str)

In [10]:
#Función para preprocesamiento
def preprocess(text):
    text = text.replace('-', ' ').lower()
    text = ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    return ' '.join(tokens)

In [18]:
#Crea una función para realizar el análisis de sentimiento con vaderSentiment y comparar los resultados:
def analizar_sentimiento(comentarios, preprocesar=True):
    analyzer = SentimentIntensityAnalyzer()
    resultados = []

    for comentario in comentarios:
        texto = comentario['Comentario']

        if preprocesar:
            texto = preprocess(texto)

        # Realizar análisis de sentimiento
        sentiment = analyzer.polarity_scores(texto)

        # Determinar la etiqueta basada en el análisis
        if sentiment['compound'] >= 0.05:
            sentimiento_predicho = 'Positive'
        elif sentiment['compound'] <= -0.05:
            sentimiento_predicho = 'Negative'
        else:
            sentimiento_predicho = 'Neutral'

        # Guardar el resultado en una lista
        resultados.append({
            'Comentario': texto,
            'Sentimiento_Real': comentario['Sentimiento'],
            'Sentimiento_Predicho': sentimiento_predicho
        })

    return resultados

In [19]:
#Generar grupos aleatorios de comentarios y realizar el análisis de sentimiento para cada grupo
random.seed(42)  # Establece una semilla para reproducibilidad
grupos = {
    5: random.sample(df.to_dict(orient='records'), 5),
    10: random.sample(df.to_dict(orient='records'), 10),
    15: random.sample(df.to_dict(orient='records'), 15),
    20: random.sample(df.to_dict(orient='records'), 20)
}

resultados_totales = {}

for tamano_grupo, comentarios_grupo in grupos.items():
    resultados_sin_preprocesar = analizar_sentimiento(comentarios_grupo, preprocesar=False)
    resultados_con_preprocesamiento = analizar_sentimiento(comentarios_grupo, preprocesar=True)
    
    resultados_totales[tamano_grupo] = {
        'Sin_Preprocesamiento': resultados_sin_preprocesar,
        'Con_Preprocesamiento': resultados_con_preprocesamiento
    }

In [21]:
#Calcula la precisión para cada grupo y muestra los resultados
from sklearn.metrics import accuracy_score

precisones = {}

for tamano_grupo, resultados in resultados_totales.items():
    y_real = [resultado['Sentimiento_Real'] for resultado in resultados['Sin_Preprocesamiento']]
    y_pred_sin_preprocesamiento = [resultado['Sentimiento_Predicho'] for resultado in resultados['Sin_Preprocesamiento']]
    y_pred_con_preprocesamiento = [resultado['Sentimiento_Predicho'] for resultado in resultados['Con_Preprocesamiento']]
    
    precision_sin_preprocesamiento = accuracy_score(y_real, y_pred_sin_preprocesamiento)
    precision_con_preprocesamiento = accuracy_score(y_real, y_pred_con_preprocesamiento)
    
    precisones[tamano_grupo] = {
        'Sin_Preprocesamiento': precision_sin_preprocesamiento,
        'Con_Preprocesamiento': precision_con_preprocesamiento
    }

# Mostrar la tabla comparativa de precisión
tabla_comparativa = pd.DataFrame(precisones).T
tabla_comparativa

,Sin_Preprocesamiento,Con_Preprocesamiento
5,0.60,0.600000
10,0.50,0.500000
15,0.40,0.333333
20,0.65,0.600000


## **Conclusiones:**

**Sin Preprocesamiento vs. Con Preprocesamiento:**

En general, la precisión del análisis de sentimiento es similar tanto para los comentarios sin preprocesamiento como para los comentarios con preprocesamiento. En todos los tamaños de grupo (5, 10, 15 y 20 comentarios), las precisiones son bastante cercanas entre sí para ambas condiciones.

**Tamaño del Grupo:**

No parece haber una tendencia clara en la precisión en relación con el tamaño del grupo. Por ejemplo, no podemos concluir que un grupo más grande o más pequeño tenga una precisión significativamente diferente.

**Resultados Esperados:**

Las precisiones que varían entre 0.40 y 0.65 indican que el análisis de sentimiento no es perfecto, y hay margen para mejorar la precisión en ambos casos (con y sin preprocesamiento). Es importante destacar que los resultados pueden depender en gran medida de la calidad y la diversidad de los comentarios en el conjunto de datos, así como de la efectividad de la librería de análisis de sentimiento utilizada (VADER en este caso).

**Impacto del Preprocesamiento:**

Aunque el preprocesamiento completo de texto (eliminación de guiones, minúsculas, eliminación de acentos y eliminación de stopwords) tiende a tener resultados ligeramente inferiores en términos de precisión en este conjunto de datos específico, este no es necesariamente el caso en todos los conjuntos de datos. El impacto del preprocesamiento puede variar según la naturaleza y la calidad de los comentarios.